# Projeto Automação Web - Busca de Preços

### Objetivo: treinar um projeto em que a gente tenha que usar automações web com Selenium para buscar as informações que precisamos

- Já fizemos um projeto com esse objetivo no Módulo de Python e Web e em gravações de encontros ao vivo, mas não custa nada treinar mais um pouco.

### Como vai funcionar:

- Imagina que você trabalha na área de compras de uma empresa e precisa fazer uma comparação de fornecedores para os seus insumos/produtos.

- Nessa hora, você vai constantemente buscar nos sites desses fornecedores os produtos disponíveis e o preço, afinal, cada um deles pode fazer promoção em momentos diferentes e com valores diferentes.

- Seu objetivo: Se o valor dos produtos for abaixo de um preço limite definido por você, você vai descobrir os produtos mais baratos e atualizar isso em uma planilha.
- Em seguida, vai enviar um e-mail com a lista dos produtos abaixo do seu preço máximo de compra.

- No nosso caso, vamos fazer com produtos comuns em sites como Google Shopping e Buscapé, mas a ideia é a mesma para outros sites.

### Outra opção:

- APIs

### O que temos disponível?

- Planilha de Produtos, com os nomes dos produtos, o preço máximo, o preço mínimo (para evitar produtos "errados" ou "baratos de mais para ser verdade" e os termos que vamos querer evitar nas nossas buscas.

### O que devemos fazer:

- Procurar cada produto no Google Shopping e pegar todos os resultados que tenham preço dentro da faixa e sejam os produtos corretos
- O mesmo para o Buscapé
- Enviar um e-mail para o seu e-mail (no caso da empresa seria para a área de compras por exemplo) com a notificação e a tabela com os itens e preços encontrados, junto com o link de compra. (Vou usar o e-mail pythonimpressionador@gmail.com. Use um e-mail seu para fazer os testes para ver se a mensagem está chegando)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

import re

import pandas as pd
import time

navegador = webdriver.Chrome()

def tem_termo_banido(nome_produto, termos_banidos):
  tem_termo_banido = False

  for termo_banido in termos_banidos:
    if termo_banido in nome_produto.lower():
      tem_termo_banido = True
      break

  return tem_termo_banido

def tem_todos_os_itens_da_pesquisa(nome_produto, produto):
  tem_todos_os_itens_da_pesquisa = True

  for item_pesquisa in produto.split(' '):
    if not item_pesquisa in nome_produto.lower():
      tem_todos_os_itens_da_pesquisa = False
      break
  
  return tem_todos_os_itens_da_pesquisa

def pesquisa_google(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
  produto = produto.lower()
  termos_banidos = termos_banidos.lower().split(' ')
  preco_minimo = float(preco_minimo)
  preco_maximo = float(preco_maximo)

  lista_produtos = []

  navegador.get('https://www.google.com/shopping?hl=pt-BR')

  # Expressão regular para capturar somente o preço
  padrao = re.compile(r"R\$ \d{1,3}(\.\d{3})*,\d{2}")

  navegador.find_element(By.ID, 'APjFqb').send_keys(f'{produto} entre R$ {preco_minimo} e R$ {preco_maximo}', Keys.ENTER)

  # esperar o captcha
  WebDriverWait(navegador, 20).until(EC.presence_of_element_located((By.CLASS_NAME, 'I8iMf')))
  time.sleep(1)

  # scroll na tela
  for i in range(4):
    time.sleep(2)
    navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")

  elementos = navegador.find_elements(By.CLASS_NAME, 'I8iMf')

  for elemento in elementos:
    nome_produto = elemento.find_element(By.CLASS_NAME, 'gkQHve.SsM98d.RmEs5b').text

    if tem_termo_banido(nome_produto, termos_banidos):
      continue
    
    if not tem_todos_os_itens_da_pesquisa(nome_produto, produto):
      continue

    preco_produto = elemento.find_element(By.CLASS_NAME, 'lmQWe').text

    # Extraindo o valor
    resultado = padrao.search(preco_produto)

    # Se encontrar, retorna o valor; caso contrário, retorna None
    preco_produto = resultado.group() if resultado else None
    
    preco_tratado = float(preco_produto.replace('R$ ', '').replace('.', '').replace(',','.'))
    
    if preco_tratado >= preco_minimo and preco_tratado <= preco_maximo:
      clickable = elemento.find_element(By.CLASS_NAME, "MtXiu")
      ActionChains(navegador).click(clickable).perform()
      time.sleep(2)
      link = navegador.find_element(By.CLASS_NAME, 'PshwNb').find_element(By.TAG_NAME, 'a').get_attribute('href')
      lista_produtos.append((nome_produto, preco_produto, link))

  return lista_produtos

def pesquisa_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo):
  produto = produto.lower()
  termos_banidos = termos_banidos.lower().split(' ')
  preco_minimo = float(preco_minimo)
  preco_maximo = float(preco_maximo)

  lista_produtos = []

  navegador.get('https://www.buscape.com.br/')

  # Expressão regular para capturar somente o preço
  padrao = re.compile(r"R\$ \d{1,3}(\.\d{3})*,\d{2}")

  navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(f'{produto}', Keys.ENTER)

  elementos = navegador.find_elements(By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh')

  for elemento in elementos:
    nome_produto = elemento.find_element(By.TAG_NAME, 'h2').text
    
    if tem_termo_banido(nome_produto, termos_banidos):
      continue
    
    if not tem_todos_os_itens_da_pesquisa(nome_produto, produto):
      continue

    preco_produto = elemento.find_element(By.TAG_NAME, 'p').text

    # Extraindo o valor
    resultado = padrao.search(preco_produto)

    # Se encontrar, retorna o valor; caso contrário, retorna None
    preco_produto = resultado.group() if resultado else None
    
    preco_tratado = float(preco_produto.replace('R$ ', '').replace('.', '').replace(',','.'))
    
    if preco_tratado >= preco_minimo and preco_tratado <= preco_maximo:
      link = elemento.get_attribute('href')
      lista_produtos.append((nome_produto, preco_produto, link))

  return lista_produtos

# produto = 'iphone 12 64 gb'
# termos_banidos = 'mini watch pelicula capa'
# preco_minimo = 2000
# preco_maximo = 3500

# print(pesquisa_google(navegador, produto, termos_banidos, preco_minimo, preco_maximo))
# print(pesquisa_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo))

buscas_df = pd.read_excel('buscas.xlsx')

lista_df = pd.DataFrame()

for i in buscas_df.index:
  produto = buscas_df.loc[i]['Nome']
  termos_banidos = buscas_df.loc[i]['Termos banidos']
  preco_minimo = buscas_df.loc[i]['Preço mínimo']
  preco_maximo = buscas_df.loc[i]['Preço máximo']

  lista_google = pesquisa_google(navegador, produto, termos_banidos, preco_minimo, preco_maximo)
  
  if lista_google :
    lista_google_df = pd.DataFrame(lista_google, columns=["nome", "preco", "link"])
    lista_df = pd.concat([lista_df, lista_google_df])

  lista_buscape = pesquisa_buscape(navegador, produto, termos_banidos, preco_minimo, preco_maximo)

  if lista_buscape :
    lista_buscape_df = pd.DataFrame(lista_buscape, columns=["nome", "preco", "link"])
    lista_df = pd.concat([lista_df, lista_buscape_df])

display(lista_df)
navegador.quit()

KeyboardInterrupt: 

# Exportando como arquivo

In [ ]:
lista_df = lista_df.iloc[:, 1:]  # Remove a primeira coluna, caso seja o índice
lista_df.to_excel("Ofertas.xlsx", index=False)